In [7]:
import pandas as pd
import pickle

In [2]:
y_all = pd.read_csv('data/y_ugr_all.csv', index_col=0)

In [11]:
totals = ['NO_TOTAL', 'TOTAL']
years_choices = [('~2014', (2014, 2015, 2016, 2017)), ('~2016', (2016, 2017)), ]
features_org = {}
features_all = {}

for total_or_not in totals:
    for year_type, years in years_choices:
        ftr = pd.read_csv('data/x_%s_%s.csv' % (total_or_not, year_type), index_col=0)
        cols_by_type = pickle.load(open('data/x_%s_%s_cols_by_type.pkl' % (total_or_not, year_type), 'rb'))
        features_org[(total_or_not, year_type)] = (ftr, cols_by_type)
        features_all[(total_or_not, year_type)] = (ftr.loc[y_all.index], cols_by_type)

In [ ]:
test_y = y_all.ugr

for seed in [0, 100, 972, 5258, 7821, 40918, 57852, 168352, 291592, 789729423]:
    for frac in [0.2, 0.4, 0.6, 0.8]:
        exp_path = 'data/downsample_experiment/seed_%d/frac_%.2f' % (seed, frac)
        mkdirs_if_not_exist(exp_path)
        print(dtm.now(), 'experiment top dir =', exp_path)
        y_frac = pd.read_csv('data/downsample_experiment/seed_%d/y_ugr_all_frac_%.2f.csv' % (seed, frac))
        train_y = y_frac.ugr
        
        for total_or_not in ['NO_TOTAL', 'TOTAL']:
            for year_type, years in [('~2014', (2014, 2015, 2016, 2017)), ('~2016', (2016, 2017)), ]:

                print(dtm.now(), 'loading features', total_or_not, year_type)
                ftr_org, cols_by_type = features_org[(total_or_not, year_type)]
                train_x = ftr_org.loc[train_y.index]
                test_x = features_all[(total_or_not, year_type)][0]
                ftr_name = train_x.columns
                
                max_cut_cols = get_max_cut_cols(cols_by_type)
                
                for max_cut in (True, False,):
                    for selection_type in ['None', 'rfecv_linsvc', 'mrmr']:
                        exp_param = '#'.join(
                            [total_or_not, year_type, 'max_cut' if max_cut else 'minmax_only', selection_type])
                        exp_param_path = '%s/%s' % (exp_path, exp_param)
                        mkdirs_if_not_exist(exp_param_path)
                        print(dtm.now(), 'experiment with', exp_param)
                        gridcv_ready = scale_and_selection(train_x, train_y, test_x, test_y, selection_type, max_cut)
                        grid_eval(gridcv_ready, exp_param_path, ftr_name)

In [15]:
ftr.shape

(13522, 64)